In [ ]:
import requests, re, pandas as pd
from bs4 import BeautifulSoup

def get_data():
    url='https://www.dol.gov/agencies/eta/foreign-labor/performance'
    r=requests.get(url)
    soup=BeautifulSoup(r.content,'html.parser')
    res=soup.select('table.cust_table')
    final_links=[]
    keywords=['Appendix', 'Worksites', 'Icert']
    for i in res:
        if i.get('summary') == 'LCA Programs (H-1B, H-1B1, E-3)':
            i= i.find_all('a')
            for j in i:
                j=j.get('href')
                # final_links.append(j)
                excel_sheet = re.search('.xlsx',j)
                key_search = re.search("|".join(re.escape(word) for word in keywords), j)
                if  excel_sheet is not None and key_search is None :
                    final_links.append(j)
    return final_links


def download_all_excel_files(final_links):
# URL of the Excel file
    for i in final_links:
        url = f'https://www.dol.gov{i}'
        response = requests.get(url)
        check = re.search(r'pdfs\/(.*)',i)
        checked = check.group().replace('pdfs/','')   
        # dbutils.fs.put(f'/mnt/files/dol-support-files/{checked}','')       
        if response.status_code == 200:
            with open(f'/dbfs/mnt/files/dol-support-files/{checked}', 'wb') as excel_file:
                excel_file.write(response.content)
            display(len(dbutils.fs.ls('/mnt/files/dol-support-files/')))
        else:
            print("Failed to download the file. Status code:", response.status_code, 'for file ', checked)
        # break

final_links = get_data()
result = download_all_excel_files(final_links)
